In [1]:
import uproot
import numpy as np
import awkward as ak

import vector

import hist
import correctionlib

def spacial_invert(vec):
    return -vec.to_Vector3D().to_Vector4D()+vector.obj(px=0, py=0, pz=0, E=vec.E)

def load_corrector(file,):
    if file.endswith(".json.gz"):
        import gzip
        with gzip.open(file, 'rt') as file:
            data = file.read().strip()
            ceval = correctionlib.CorrectionSet.from_string(data)
    else:
        ceval = correctionlib.CorrectonSet.from_file(file)
    return ceval


In [2]:
filter_names = ['/(Electron|Muon|Jet)_(pt|eta|phi|mass)/',
                '/n(Muon|Electron|Jet)/',
                '/(Electron|Muon)_charge/',
                '/Muon_(triggerIdLoose)/',
                '/Muon_(pfRelIso04_all|tightId)/',
                '/HLT_(IsoMu24|Ele32_WPTight_Gsf)/',
                'Electron_mvaFall17V2Iso_WP90',
                '/Jet_(btagDeepB|btagDeepFlavB)/',
                'Pileup_nTrueInt',
                'genWeight']
mc_filter_names = ['/(Electron|Muon)_genPart(Idx|Flav)/',
                '/GenPart_(pdgId|genPartIdxMother)/',
                'nGenPart']

In [3]:

correctionfiles = {
    'muon': "../corrections/muon_Z.json.gz",
    'electron': "../corrections/electron.json.gz",
    'pileup': "../corrections/puWeights.json.gz",
    'jets': "../corrections/jet_jerc.json.gz",
    'b_tag': "../corrections/btagging.json.gz"
}

outfile = uproot.recreate('outfile.root')
outfile.mktree("tout", {'Muon_pt': np.float64,
 'Muon_eta': np.float64,
 'Muon_phi': np.float64,
 'Muon_mass': np.float64,
 'Electron_pt':  np.float64,
 'Electron_eta': np.float64,
 'Electron_phi': np.float64,
 'Electron_mass': np.float64,
 'Jet_pt': np.float64,
 'Jet_eta': np.float64,
 'Jet_phi': np.float64,
 'Jet_mass': np.float64,
 'mu_e_inv_mass': np.float64,
 'leading_lepton_pt': np.float64,
 'N_jet_loose': np.int32,
 'N_jet_tight':np.int32,
 'N_jet_medium':np.int32,
})
## define histograms

h_Muon_pt = hist.Hist(hist.axis.Regular(bins=100,start=0,stop=200,name="Muon pt"))
h_Muon_eta = hist.Hist(hist.axis.Regular(bins=100,start=-5,stop=5,name="Muon eta"))
h_Electron_pt = hist.Hist(hist.axis.Regular(bins=100,start=0,stop=200, name="Electron pt",)) 
h_Electron_eta = hist.Hist(hist.axis.Regular(bins=100,start=-5,stop=5, name="Electron eta",))

h_Muon_pt_weighted = hist.Hist(hist.axis.Regular(bins=100,start=0,stop=200,name="Muon pt"))
h_Muon_eta_weighted = hist.Hist(hist.axis.Regular(bins=100,start=-5,stop=5,name="Muon eta"))
h_Electron_pt_weighted = hist.Hist(hist.axis.Regular(bins=100,start=0,stop=200,name="Electron pt"))
h_Electron_eta_weighted = hist.Hist(hist.axis.Regular(bins=100,start=-5,stop=5,name="Electron eta"))

h_Muon_Electron_invariant_mass = hist.Hist(hist.axis.Regular(bins=100,start=12,stop= 412,name="Muon Electron Inv. Mass"))
h_Muon_Electron_invariant_mass_weighted = hist.Hist(hist.axis.Regular(bins=100,start=12,stop= 412,name="Muon Electron Inv. Mass"))

h_leading_lepton_pt = hist.Hist(hist.axis.Regular(bins=45,start=20,stop=200,name="leading lepton pt"))
h_leading_lepton_pt_weighted = hist.Hist(hist.axis.Regular(bins=45,start=20,stop=200,name="leading lepton pt"))


In [4]:
outfile.classnames()

{'tout;1': 'TTree'}

In [6]:

#filename = "../input/Data/EGamma/Run2018A-UL2018_MiniAODv2_NanoAODv9-v1/00B7FFB1-3455-C941-AE3B-CF7085966A41.root"
#filename= "../input/MC/DY/DYJetsToLL_1J_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v15_L1v1-v1/130000/6E2A3FD3-C3FD-5A4C-B97C-58DC2C0A0A60.root"
filename = "../input/MC/Signal/019426EE-3D50-1249-B266-F6DBA0AFE3B5.root"

mc_flag = True
file = uproot.open(filename)
tree = file['Events']
#def skimming(tree, mc_flag, lumi=None, xs=None):
    # ^ is xor in python
trigger_cut = "HLT_IsoMu24 | HLT_Ele32_WPTight_Gsf"
btag_deepflav_wp = {'loose': 0.0490, 'medium': 0.2783, 'tight': 0.71}
# get the sumW from the runs tree
if mc_flag:
    n_gen = file['Runs']['genEventCount'].array()
    Sum_W = file['Runs']['genEventSumw'].array()
for events in tree.iterate(
    filter_name=filter_names+mc_filter_names,cut = trigger_cut,
    entry_stop=100000):
    ## apply muon and electron cuts 
    m_pt_cut = events['Muon_pt']>27
    m_eta_cut = np.abs(events['Muon_eta'])<2.4
    m_iso_cut = events['Muon_pfRelIso04_all']<0.15
    m_tight_cut = events["Muon_tightId"]

    muon_cuts = (m_pt_cut) &(m_eta_cut) & (m_iso_cut)&m_tight_cut
    events['muon_cuts'] = muon_cuts
    events = events[ak.any(muon_cuts, axis=1)]

    e_pt_cut =  events['Electron_pt']>35
    e_eta_cut = np.abs(events['Electron_eta'])<2.4
    e_iso_cut =  events["Electron_mvaFall17V2Iso_WP90"]

    electron_cuts = (e_pt_cut)&(e_eta_cut)&(e_iso_cut)
    events['electron_cuts'] = electron_cuts
    events = events[ak.any(electron_cuts, axis=1)]

    b_cut = events['Jet_btagDeepFlavB']>btag_deepflav_wp['medium']
    b_tag_eta = events['Jet_eta']
    events = events[ak.any(b_cut, axis=1)]
    # make sure that muon and electron are of opposite charge
    # this cut is applied after the others because you can only 
    # multiply two arrays if they both have at least one entry along the first dim
    # the pt cuts make sure of this: we only have events left where we have 
    # at least one electron and at leats one muon
    charge_cut = (events['Muon_charge', events['muon_cuts']][:, 0] * events['Electron_charge', events['electron_cuts']][:, 0]) < 0
    events = events[charge_cut]
    # create four-vectors
    events['Muon_pt'] = events['Muon_pt', events['muon_cuts']][:, 0]
    events['Muon_eta'] = events['Muon_eta', events['muon_cuts']][:, 0]
    events['Muon_phi'] = events['Muon_phi', events['muon_cuts']][:, 0]
    events['Muon_mass'] = events['Muon_mass', events['muon_cuts']][:, 0]
    events['Electron_pt'] = events['Electron_pt', events['electron_cuts']][:, 0]
    events['Electron_eta'] = events['Electron_eta', events['electron_cuts']][:, 0]
    events['Electron_phi'] = events['Electron_phi', events['electron_cuts']][:, 0]
    events['Electron_mass'] = events['Electron_mass', events['electron_cuts']][:, 0]
    # objects are ordered by pt. If there was more than one mu/e that passed all
    # requirements we take index 0 for highest pt
    muon_4d = vector.array({'pt':events['Muon_pt'],
                            'eta': events['Muon_eta'], 
                            'phi':events['Muon_phi'],
                            'mass': events['Muon_mass']})
    electron_4d = vector.array({'pt':events['Electron_pt'],
                                'eta': events['Electron_eta'],
                                'phi':events['Electron_phi'],
                                'mass': events['Electron_mass']})
    # calculate the deltaR
    # cut out events with deltaR < 0.4
    delta_r_cut = muon_4d.deltaR(electron_4d) > 0.4
    muon_4d = muon_4d[delta_r_cut]; electron_4d=electron_4d[delta_r_cut]
    events = events[delta_r_cut]
    # how many jets would we have if we took other wp's?
    events['N_jet_loose'] = ak.sum(events['Jet_btagDeepFlavB']>btag_deepflav_wp['loose'], axis=1)
    events['N_jet_medium'] = ak.sum(events['Jet_btagDeepFlavB']>btag_deepflav_wp['medium'], axis=1)
    events['N_jet_tight'] = ak.sum(events['Jet_btagDeepFlavB']>btag_deepflav_wp['tight'], axis=1)
    # first drop events where there is no jet that passes the wp
    b_tag_medium_cut = events['Jet_btagDeepFlavB']>btag_deepflav_wp['medium']
    b_tag_eta = np.abs(events['Jet_eta'])<2.4
    events['b_tag_cut'] = b_tag_medium_cut&b_tag_eta
    events = events[ak.any(events['b_tag_cut'], axis=1)]
    muon_4d=muon_4d[ak.any(events['b_tag_cut'], axis=1)]
    electron_4d=electron_4d[ak.any(events['b_tag_cut'], axis=1)]
    # now our event has at least one jet passing the wp
    # event could look like this: [not_passing, not_passing, passing, not_passing, passing]
    # we want the first one that passes (index = 2)

    events['Jet_pt'] = events['Jet_pt', events['b_tag_cut']][:, 0]
    events['Jet_eta'] = events['Jet_eta', events['b_tag_cut']][:, 0]
    events['Jet_phi'] = events['Jet_phi', events['b_tag_cut']][:, 0]
    events['Jet_mass'] = events['Jet_mass', events['b_tag_cut']][:, 0]
    events['Jet_btagDeepFlavB'] = events['Jet_btagDeepFlavB', events['b_tag_cut']][:, 0]
    jet_4d = vector.array({'pt':events['Jet_pt'],
                            'eta': events['Jet_eta'],
                            'phi': events['Jet_phi'],
                            'mass': events['Jet_mass'],})
    #opposite_jet = spacial_invert(jet_4d)
    dR_mu_jet = muon_4d.deltaR(jet_4d)
    dR_e_jet = electron_4d.deltaR(jet_4d)
    dr_cut = (dR_mu_jet>0.4)&(dR_e_jet>0.4)
    events=events[dr_cut]
    muon_4d=muon_4d[dr_cut]
    electron_4d = electron_4d[dr_cut]
    jet_4d = jet_4d[dr_cut]
    #dR_mu_e = muon_4d.deltaR(electron_4d)

    ## coreections
    if mc_flag:
        muon_eval = load_corrector(correctionfiles['muon'])
        mu_c_trigger = muon_eval['NUM_IsoMu24_DEN_CutBasedIdTight_and_PFIsoTight'].evaluate(
            '2018_UL', np.abs(muon_4d.eta), muon_4d.pt, 'sf')
        #mu_c_reco = muon_eval['NUM_GlobalMuons_DEN_genTracks'].evaluate(
        #    '2018_UL', np.abs(muon_4d.eta), muon_4d.pt, 'sf')
        mu_c_id = muon_eval['NUM_TightID_DEN_genTracks'].evaluate(
            '2018_UL', np.abs(muon_4d.eta), muon_4d.pt, 'sf')
        mu_c_iso = muon_eval['NUM_TightRelIso_DEN_TightIDandIPCut'].evaluate(
            '2018_UL', np.abs(muon_4d.eta), muon_4d.pt, 'sf')
        muon_c = mu_c_trigger*mu_c_id*mu_c_iso
        electron_eval = load_corrector(correctionfiles['electron'])
        ele_c = electron_eval['UL-Electron-ID-SF'].evaluate(
            '2018', 'sf', 'Tight', electron_4d.eta, electron_4d.pt
        )
        pu_eval = load_corrector(correctionfiles['pileup'])
        pu_c = pu_eval['Collisions18_UltraLegacy_goldenJSON'].evaluate(events['Pileup_nTrueInt'], 'nominal')
        #jet_eval = load_corrector(correctionfiles['jets'])
        #jet_c = jet_eval['deepJet_mujets'].evaluate(
        b_tag_eval = load_corrector(correctionfiles['b_tag'])
        b_tag_c_wp = b_tag_eval['deepJet_mujets'].evaluate(
            'central', "M", 5, np.abs(jet_4d.eta), jet_4d.pt
        )
        b_tag_c_shape = b_tag_eval['deepJet_shape'].evaluate(
            'central', 5, np.abs(jet_4d.eta), jet_4d.pt, events['Jet_btagDeepFlavB']
        )
        b_tag_c = b_tag_c_shape*b_tag_c_wp
        corrections = muon_c * ele_c * pu_c * b_tag_c
        
        weight = events['genWeight']*corrections

    #if mc_flag:
        #weight = events['genWeight']*lumi*xs


    # fill the histograms 
    h_Muon_pt.fill(muon_4d.pt)
    h_Muon_eta.fill(muon_4d.eta)
    h_Electron_pt.fill(electron_4d.pt)
    h_Electron_eta.fill(electron_4d.eta)

    events['mu_e_inv_mass'] = (muon_4d + electron_4d).M
    events['leading_lepton_pt'] = np.max([muon_4d.pt,electron_4d.pt], axis=0)
    h_Muon_Electron_invariant_mass.fill(events['mu_e_inv_mass'],)
    h_leading_lepton_pt.fill(events['leading_lepton_pt'])

    tout_dict = {i: events[i] for i in events[0].fields if i.endswith(('pt', 'eta', 
    'mass', 'phi', 'medium', 'loose', 'tight'))}
    outfile['tout'].extend(tout_dict)
    outfile['h_Muon_pt'] = h_Muon_pt
    outfile['h_Muon_eta'] = h_Muon_eta
    outfile['h_Electron_pt'] = h_Electron_pt
    outfile['h_Electron_eta'] = h_Electron_eta
    outfile['h_Muon_Electron_invariant_mass'] = h_Muon_Electron_invariant_mass
    outfile['h_leading_lepton_pt'] = h_leading_lepton_pt

    if mc_flag:
        h_Muon_Electron_invariant_mass_weighted.fill(events['mu_e_inv_mass'],weight=weight)
        h_leading_lepton_pt_weighted.fill(events['leading_lepton_pt'], weight=weight)

        h_Muon_eta_weighted.fill(muon_4d.eta, weight = weight)
        h_Muon_pt_weighted.fill(muon_4d.pt, weight = weight)
        h_Electron_pt_weighted.fill(electron_4d.pt, weight = weight)
        h_Electron_eta_weighted.fill(electron_4d.eta, weight = weight)
        outfile['h_Muon_pt_weighted'] = h_Muon_pt_weighted
        outfile['h_Muon_eta_weighted'] = h_Muon_eta_weighted
        outfile['h_Electron_pt_weighted'] = h_Electron_pt_weighted
        outfile['h_Muon_eta_weighted'] = h_Muon_eta_weighted
        outfile['h_Muon_Electron_invariant_mass_weighted'] = h_Muon_Electron_invariant_mass_weighted
        outfile['h_leading_lepton_pt_weighted'] = h_leading_lepton_pt_weighted





ValueError: cannot fit jagged slice with length 6167 into RegularArray of size 6103

(https://github.com/scikit-hep/awkward-1.0/blob/1.8.0/src/libawkward/array/RegularArray.cpp#L1533)

In [6]:
Sum_W

<Array [9.99e+07] type='1 * float64'>

In [9]:
tout_dict

{'Muon_pt': <Array [77.7, 29, 111, ... 64.6, 38.5, 35.9] type='5997 * float32'>,
 'Muon_eta': <Array [0.316, 0.887, -0.746, ... -1.62, -1.33] type='5997 * float32'>,
 'Muon_phi': <Array [-1.42, 0.642, -2.18, ... 0.0312, 2.27] type='5997 * float32'>,
 'Muon_mass': <Array [0.106, 0.106, 0.106, ... 0.106, 0.106] type='5997 * float32'>,
 'Electron_pt': <Array [39.9, 71.6, 76.3, ... 37.3, 65.8, 61.7] type='5997 * float32'>,
 'Electron_eta': <Array [-0.702, 0.355, 0.084, ... -1.36, 0.912] type='5997 * float32'>,
 'Electron_phi': <Array [-0.885, -2.8, 0.84, ... 0.691, -0.722] type='5997 * float32'>,
 'Electron_mass': <Array [-0.0129, -0.0149, ... 0.0246, -0.0191] type='5997 * float32'>,
 'Jet_pt': <Array [122, 84.8, 153, ... 28.3, 40.5, 144] type='5997 * float32'>,
 'Jet_eta': <Array [-0.746, -0.545, ... -2.29, 0.699] type='5997 * float32'>,
 'Jet_phi': <Array [-1.89, -0.249, 0.205, ... 2.51, 0.122] type='5997 * float32'>,
 'Jet_mass': <Array [11.1, 9.15, 23.5, ... 6.85, 9.05, 24.6] type='599

In [7]:
tout_dict

{'Muon_pt': <Array [77.7, 29, 111, ... 64.6, 38.5, 35.9] type='5997 * float32'>,
 'Muon_eta': <Array [0.316, 0.887, -0.746, ... -1.62, -1.33] type='5997 * float32'>,
 'Muon_phi': <Array [-1.42, 0.642, -2.18, ... 0.0312, 2.27] type='5997 * float32'>,
 'Muon_mass': <Array [0.106, 0.106, 0.106, ... 0.106, 0.106] type='5997 * float32'>,
 'Electron_pt': <Array [39.9, 71.6, 76.3, ... 37.3, 65.8, 61.7] type='5997 * float32'>,
 'Electron_eta': <Array [-0.702, 0.355, 0.084, ... -1.36, 0.912] type='5997 * float32'>,
 'Electron_phi': <Array [-0.885, -2.8, 0.84, ... 0.691, -0.722] type='5997 * float32'>,
 'Electron_mass': <Array [-0.0129, -0.0149, ... 0.0246, -0.0191] type='5997 * float32'>,
 'Jet_pt': <Array [122, 84.8, 153, ... 28.3, 40.5, 144] type='5997 * float32'>,
 'Jet_eta': <Array [-0.746, -0.545, ... -2.29, 0.699] type='5997 * float32'>,
 'Jet_phi': <Array [-1.89, -0.249, 0.205, ... 2.51, 0.122] type='5997 * float32'>,
 'Jet_mass': <Array [11.1, 9.15, 23.5, ... 6.85, 9.05, 24.6] type='599

In [151]:
tout_dict = {i: events[i] for i in events[0].fields if i.endswith(('pt', 'eta', 'mass', 'phi', 'charge'))}

In [152]:
tout_dict

{'Electron_eta': <Array [[-0.702], [0.355, ... -1.36], [0.912]] type='5997 * var * float32'>,
 'Electron_mass': <Array [[-0.0129], [-0.0149, ... [-0.0191]] type='5997 * var * float32'>,
 'Electron_phi': <Array [[-0.885], [-2.8, ... 0.691], [-0.722]] type='5997 * var * float32'>,
 'Electron_pt': <Array [[39.9], [71.6, 41], ... [65.8], [61.7]] type='5997 * var * float32'>,
 'Electron_charge': <Array [[1], [-1, 1], [1, ... [1], [-1], [-1]] type='5997 * var * int32'>,
 'Jet_eta': <Array [[-2.26, -0.746, ... 0.0683, 2.61]] type='5997 * var * float32'>,
 'Jet_mass': <Array [[27, 11.1, 14.7, ... 4.48, 3.66, 4.06]] type='5997 * var * float32'>,
 'Jet_phi': <Array [[1.61, -1.89, -1.25, ... -1.93, 0.435]] type='5997 * var * float32'>,
 'Jet_pt': <Array [[299, 122, 95.5, ... 21.3, 16.2, 15.3]] type='5997 * var * float32'>,
 'Muon_eta': <Array [[0.316, -0.228], ... [-1.33, -1.21]] type='5997 * var * float32'>,
 'Muon_mass': <Array [[0.106, 0.106], ... [0.106, 0.106]] type='5997 * var * float32'>,


In [89]:
events['Jet_pt', 0]

<Array [299, 122, 95.5, ... 22.4, 19.5, 17.2] type='9 * float32'>

In [82]:
ak.num(b_tag_cut)

ValueError: 'axis' out of range for 'num'

(https://github.com/scikit-hep/awkward-1.0/blob/1.8.0/src/libawkward/array/NumpyArray.cpp#L1598)

In [62]:
np.abs(events['Jet_eta'])

<Array [[2.26, 0.746, 0.156, ... 0.0683, 2.61]] type='6071 * var * float32'>

In [65]:
ak.abs(ak.Array([1.1, 2.2, ], [3.3], [4.4, 5.5],))

AttributeError: module 'awkward' has no attribute 'abs'

In [56]:
b_tag_eval['deepJets_mujets'].evaluate(
        'central', "M", 5, np.abs(jet_eta), jet_pt
)

IndexError: map::at

In [61]:
np.abs(jet_eta)

<Array [0.746, 0.545, 0.178, ... 2.29, 0.699] type='6071 * float32'>

In [58]:
list(b_tag_eval.keys())

['deepCSV_comb',
 'deepCSV_incl',
 'deepCSV_mujets',
 'deepCSV_shape',
 'deepJet_comb',
 'deepJet_incl',
 'deepJet_mujets',
 'deepJet_shape']

In [60]:
corr = b_tag_eval['deepJet_mujets']
for ix in corr.inputs:
    print(f"   Input {ix.name} ({ix.type}): {ix.description}")

   Input systematic (string): 
   Input working_point (string): L/M/T
   Input flavor (int): hadron flavor definition: 5=b, 4=c, 0=udsg
   Input abseta (real): 
   Input pt (real): 


In [50]:
np.max([muon_4d.pt,electron_4d.pt], axis=0)

array([ 77.72613525,  71.59844971, 110.85992432, ...,  64.61579895,
        65.81275177,  61.7100296 ])

In [47]:
(muon_4d+electron_4d).M

array([ 65.9939266 ,  93.38278621, 199.6779943 , ..., 118.68889819,
        35.10983273, 159.49283047])

In [39]:
electron_4d.eta

array([-0.70166016,  0.35491943,  0.08398438, ...,  1.86621094,
       -1.35913086,  0.91186523])

In [35]:
filename = "../input/MC/Signal/019426EE-3D50-1249-B266-F6DBA0AFE3B5.root"

file = uproot.open(filename)
tree = file['Events']
skimming(tree =tree, mc_flag=True)

ValueError: in ListOffsetArray64, cannot convert to RegularArray because subarray lengths are not regular

(https://github.com/scikit-hep/awkward-1.0/blob/1.8.0/src/cpu-kernels/awkward_ListOffsetArray_toRegularArray.cpp#L22)

In [33]:
mu_eta

NameError: name 'mu_eta' is not defined

In [25]:
e_eval = load_corrector(correctionfiles['electron'])
for corr in e_eval.values():
    print(f"Correction {corr.name} has {len(corr.inputs)} inputs")
    for ix in corr.inputs:
        print(f"   Input {ix.name} ({ix.type}): {ix.description}")

Correction UL-Electron-ID-SF has 5 inputs
   Input year (string): year/scenario: example 2016preVFP, 2017 etc
   Input ValType (string): sf/sfup/sfdown (sfup = sf + syst, sfdown = sf - syst) 
   Input WorkingPoint (string): Working Point of choice : Loose, Medium etc.
   Input eta (real): supercluster eta
   Input pt (real): electron pT


In [27]:
beval = load_corrector(correctionfiles['b_tag'])

for corr in beval.values():
    print(f"Correction {corr.name} has {len(corr.inputs)} inputs")
    for ix in corr.inputs:
        print(f"   Input {ix.name} ({ix.type}): {ix.description}")

Correction deepCSV_comb has 5 inputs
   Input systematic (string): 
   Input working_point (string): L/M/T
   Input flavor (int): hadron flavor definition: 5=b, 4=c, 0=udsg
   Input abseta (real): 
   Input pt (real): 
Correction deepCSV_incl has 5 inputs
   Input systematic (string): 
   Input working_point (string): L/M/T
   Input flavor (int): hadron flavor definition: 5=b, 4=c, 0=udsg
   Input abseta (real): 
   Input pt (real): 
Correction deepCSV_mujets has 5 inputs
   Input systematic (string): 
   Input working_point (string): L/M/T
   Input flavor (int): hadron flavor definition: 5=b, 4=c, 0=udsg
   Input abseta (real): 
   Input pt (real): 
Correction deepCSV_shape has 5 inputs
   Input systematic (string): 
   Input flavor (int): hadron flavor definition: 5=b, 4=c, 0=udsg
   Input abseta (real): 
   Input pt (real): 
   Input discriminant (real): deepCSV output value
Correction deepJet_comb has 5 inputs
   Input systematic (string): 
   Input working_point (string): L/M/T
  

In [29]:
tree = file['Events']


name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
run                  | uint32_t                 | AsDtype('>u4')
luminosityBlock      | uint32_t                 | AsDtype('>u4')
event                | uint64_t                 | AsDtype('>u8')
HTXS_Higgs_pt        | float                    | AsDtype('>f4')
HTXS_Higgs_y         | float                    | AsDtype('>f4')
HTXS_stage1_1_cat... | int32_t                  | AsDtype('>i4')
HTXS_stage1_1_cat... | int32_t                  | AsDtype('>i4')
HTXS_stage1_1_fin... | int32_t                  | AsDtype('>i4')
HTXS_stage1_1_fin... | int32_t                  | AsDtype('>i4')
HTXS_stage1_2_cat... | int32_t                  | AsDtype('>i4')
HTXS_stage1_2_cat... | int32_t                  | AsDtype('>i4')
HTXS_stage1_2_fin... | int32_t                  | AsDtype('>i4')
HTXS_stage1_2_fin... | int32_t                  | AsDtype(

In [21]:
ceval = load_corrector(correctionfiles['jets'])
list(ceval.keys())

['Summer19UL18_JRV2_MC_PtResolution_AK4PFchs',
 'Summer19UL18_JRV2_MC_ScaleFactor_AK4PFchs',
 'Summer19UL18_RunA_V5_DATA_L1FastJet_AK4PFchs',
 'Summer19UL18_RunA_V5_DATA_L2L3Residual_AK4PFchs',
 'Summer19UL18_RunA_V5_DATA_L2Relative_AK4PFchs',
 'Summer19UL18_RunA_V5_DATA_L3Absolute_AK4PFchs',
 'Summer19UL18_RunB_V5_DATA_L1FastJet_AK4PFchs',
 'Summer19UL18_RunB_V5_DATA_L2L3Residual_AK4PFchs',
 'Summer19UL18_RunB_V5_DATA_L2Relative_AK4PFchs',
 'Summer19UL18_RunB_V5_DATA_L3Absolute_AK4PFchs',
 'Summer19UL18_RunC_V5_DATA_L1FastJet_AK4PFchs',
 'Summer19UL18_RunC_V5_DATA_L2L3Residual_AK4PFchs',
 'Summer19UL18_RunC_V5_DATA_L2Relative_AK4PFchs',
 'Summer19UL18_RunC_V5_DATA_L3Absolute_AK4PFchs',
 'Summer19UL18_RunD_V5_DATA_L1FastJet_AK4PFchs',
 'Summer19UL18_RunD_V5_DATA_L2L3Residual_AK4PFchs',
 'Summer19UL18_RunD_V5_DATA_L2Relative_AK4PFchs',
 'Summer19UL18_RunD_V5_DATA_L3Absolute_AK4PFchs',
 'Summer19UL18_V5_MC_AbsoluteFlavMap_AK4PFchs',
 'Summer19UL18_V5_MC_AbsoluteMPFBias_AK4PFchs',
 'Summe

In [ ]:
jet_ceval = load_corrector(correctionfiles[''])

In [43]:
list(ceval.keys())

['Collisions18_UltraLegacy_goldenJSON']

In [44]:
for corr in ceval.values():
    print(f"Correction {corr.name} has {len(corr.inputs)} inputs")
    for ix in corr.inputs:
        print(f"   Input {ix.name} ({ix.type}): {ix.description}")

Correction Collisions18_UltraLegacy_goldenJSON has 2 inputs
   Input NumTrueInteractions (real): Number of true interactions
   Input weights (string): nominal, up, or down


In [49]:
tree['Pileup_nTrueInt'].array()

<Array [20, 25, 30, 33, 4, ... 34, 44, 40, 41] type='1386000 * float32'>

In [50]:
ceval['Collisions18_UltraLegacy_goldenJSON'].evaluate(tree['Pileup_nTrueInt'].array(), "nominal")

array([0.94641737, 0.98739476, 0.99822759, ..., 1.08183655, 1.03718   ,
       1.04711103])

In [9]:
file.classnames()

{'tag;1': 'TObjString',
 'Events;1': 'TTree',
 'LuminosityBlocks;1': 'TTree',
 'Runs;1': 'TTree',
 'MetaData;1': 'TTree',
 'ParameterSets;1': 'TTree'}

In [20]:
file['Runs'].show()

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
run                  | uint32_t                 | AsDtype('>u4')
genEventCount        | int64_t                  | AsDtype('>i8')
genEventSumw         | double                   | AsDtype('>f8')
genEventSumw2        | double                   | AsDtype('>f8')
nLHEScaleSumw        | uint32_t                 | AsDtype('>u4')
LHEScaleSumw         | double[]                 | AsJagged(AsDtype('>f8'))
nLHEPdfSumw          | uint32_t                 | AsDtype('>u4')
LHEPdfSumw           | double[]                 | AsJagged(AsDtype('>f8'))


In [25]:
file['Runs']['genEventCount'].array()

<Array [1386000] type='1 * int64'>

In [23]:
file['Runs']['genEventSumw'].array(), file['Runs']['genEventSumw2'].array()

(<Array [9.99e+07] type='1 * float64'>, <Array [7.33e+09] type='1 * float64'>)

In [ ]:
def is_from_pdg(pdg_id, gen_idx, pdg_ids, mother_idxs):
    """
    Function to check if a generated daughter 
    particle originates from a mother particle 

    Parameters
    ----------
    pdg_id: (int) PDGId of the mother particle,
    gen_idx: (int) idx of the daughter particle,
    pdg_ids: array of pdg_ids,
    mother_idxs: array containing indices of where the
                 mother particle is located.

    Returns
    -------
    bool: is_from_pdg
    """
    # if a muon is misidentified it may have an idx of -1
    if gen_idx <= 0:
        return False
    mother_idx = mother_idxs[gen_idx]
    if pdg_ids[mother_idx] == pdg_id:
        return True
    elif pdg_ids[mother_idx] == pdg_ids[gen_idx]:
        return is_from_pdg(pdg_id, mother_idx, pdg_ids, mother_idxs)
    else:
        return False

In [47]:
tree.show("*PU*")

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
Pileup_nPU           | int32_t                  | AsDtype('>i4')


In [89]:
events['Muon_tightId']

<Array [[True], [], ... [True, False]] type='540 * var * bool'>

In [ ]:
events['Jet_']

In [3]:
datafile = "../input/Data/EGamma/Run2018A-UL2018_MiniAODv2_NanoAODv9-v1/00B7FFB1-3455-C941-AE3B-CF7085966A41.root"
dfile = uproot.open(datafile)
dfile.classnames()

{'tag;1': 'TObjString',
 'Events;1': 'TTree',
 'LuminosityBlocks;1': 'TTree',
 'Runs;1': 'TTree',
 'MetaData;1': 'TTree',
 'ParameterSets;1': 'TTree'}

In [5]:
dfile['Events'].show("*Pileup*")

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------


In [6]:
mc_file = "../input/MC/Signal/019426EE-3D50-1249-B266-F6DBA0AFE3B5.root"
mc_file = uproot.open(mc_file)
mc_file.classnames()

{'tag;1': 'TObjString',
 'Events;1': 'TTree',
 'LuminosityBlocks;1': 'TTree',
 'Runs;1': 'TTree',
 'MetaData;1': 'TTree',
 'ParameterSets;1': 'TTree'}

In [7]:
mc_file['Events'].show("*Pileup*")

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
Pileup_nTrueInt      | float                    | AsDtype('>f4')
Pileup_pudensity     | float                    | AsDtype('>f4')
Pileup_gpudensity    | float                    | AsDtype('>f4')
Pileup_nPU           | int32_t                  | AsDtype('>i4')
Pileup_sumEOOT       | int32_t                  | AsDtype('>i4')
Pileup_sumLOOT       | int32_t                  | AsDtype('>i4')
